In [24]:
print("ok")

ok


In [25]:
%pwd

'c:\\Users\\NizamiSevindi\\Documents\\Projects\\PYTHON\\End-To-End-Medical-Gen-Ai\\research'

In [26]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\NizamiSevindi\\Documents\\Projects\\PYTHON\\End-To-End-Medical-Gen-Ai'

In [27]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [28]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [29]:
extracted_data=load_pdf_file(data='Data/')

In [30]:
extracted_data

[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'TeX', 'creationdate': '2024-09-10T21:50:42+00:00', 'moddate': '2024-09-10T21:50:42+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'trapped': '/False', 'source': 'Data\\A_Comprehensive_Review_of_Low_Rank_Adaptation_in_Large_Language_Models_for_Efficient_Parameter_Tuning-1.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='A Comprehensive Review of Low-Rank\nAdaptation in Large Language Models for\nEfficient Parameter Tuning\nSeptember 10, 2024\nAbstract\nNatural Language Processing (NLP) often involves pre-training large\nmodels on extensive datasets and then adapting them for specific tasks\nthrough fine-tuning. However, as these models grow larger, like GPT-3\nwith 175 billion parameters, fully fine-tuning them becomes computa-\ntionally expensive. We propose a novel method called LoRA (Low-Rank\nAdaptation) that significantly reduces t

In [31]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [32]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 63


In [33]:
#text_chunks

In [34]:
from langchain.embeddings import HuggingFaceEmbeddings

In [35]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [36]:
embeddings = download_hugging_face_embeddings()

C:\Users\NizamiSevindi\AppData\Local\Temp\ipykernel_37740\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [37]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [33]:
# query_result

In [38]:
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [40]:
from pinecone import Pinecone, ServerlessSpec
import os
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'X-Cloud-Trace-Context': 'e18d634482fafc5b740943cd54fcf07f', 'Date': 'Thu, 13 Mar 2025 15:02:39 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [41]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [42]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [43]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [44]:
docsearch

In [45]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [46]:
retrieved_docs = retriever.invoke("What does it about?")

In [47]:
retrieved_docs

[Document(id='5884316c-b38c-4c99-b81a-614a968c8dc1', metadata={'creationdate': '2024-09-10T21:50:42+00:00', 'creator': 'TeX', 'moddate': '2024-09-10T21:50:42+00:00', 'page': 6.0, 'page_label': '7', 'producer': 'pdfTeX-1.40.26', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'source': 'Data\\A_Comprehensive_Review_of_Low_Rank_Adaptation_in_Large_Language_Models_for_Efficient_Parameter_Tuning-1.pdf', 'total_pages': 11.0, 'trapped': '/False'}, page_content='BitFit is another baseline in which only the bias parameters are updated,\nwhile all other parameters remain frozen. This method has gained attention,\nincluding in recent studies [ ?].\nPrefix-embedding tuning (PreEmbed)involves adding special tokens to\nthe input sequence, and training their embeddings. These tokens do not belong\nto the model’s original vocabulary. Their placement—either prepended (prefix)\nor appended (infix)—can significantly affect performance, as high

In [48]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [54]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [50]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [51]:
response = rag_chain.invoke({"input": "what is Lora?"})
print(response["answer"])



LoRA is a technique for reducing the storage and hardware requirements for training natural language processing models. It works by freezing most of the model's parameters and only training small rank decomposition matrices. This not only reduces the number of trainable parameters and GPU memory usage, but also maintains or even improves performance on models like RoBERTa, DeBERTa, GPT-2, and GPT-3. It also introduces no extra latency during inference, making it more efficient for practical use.


In [56]:
response = rag_chain.invoke({"input": "What is Formula?"})
print(response["answer"])



I don't know.
